# Meshtastic Weather & Sensor Analysis
Complete environmental monitoring from SenseCAP T1000-E and Meshtastic mesh network

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
session = get_active_session()
print(f"Connected: {session.get_current_account()}")

## All Sensor Data - Complete Capture

In [ ]:
all_df = session.sql("""
SELECT 
    from_id, to_id, packet_type, ingested_at,
    latitude, longitude, altitude, ground_speed, ground_track, sats_in_view, pdop, hdop, vdop, gps_timestamp,
    rx_snr, rx_rssi, hop_limit, hop_start, channel,
    battery_level, voltage, channel_utilization, air_util_tx, uptime_seconds,
    temperature, relative_humidity, barometric_pressure, gas_resistance, iaq,
    lux, white_lux, ir_lux, uv_lux, wind_direction, wind_speed, wind_gust, weight,
    pm10_standard, pm25_standard, pm100_standard, pm10_environmental, pm25_environmental, pm100_environmental, co2,
    ch1_voltage, ch1_current, ch2_voltage, ch2_current, ch3_voltage, ch3_current,
    text_message
FROM DEMO.DEMO.MESHTASTIC_DATA ORDER BY ingested_at DESC LIMIT 2000
""").to_pandas()
print(f"Records: {len(all_df)} | Columns with data: {[c for c in all_df.columns if all_df[c].notna().any()]}")
all_df.head(20)

## Weather & Environmental Sensors

In [ ]:
weather_df = session.sql("""
SELECT from_id, ingested_at, temperature, (temperature * 9/5) + 32 as temperature_f,
    relative_humidity, barometric_pressure, gas_resistance, iaq, lux, uv_lux, ir_lux, white_lux,
    wind_speed, wind_direction, wind_gust, pm10_standard, pm25_standard, pm100_standard, co2
FROM DEMO.DEMO.MESHTASTIC_DATA
WHERE temperature IS NOT NULL OR relative_humidity IS NOT NULL OR barometric_pressure IS NOT NULL 
    OR lux IS NOT NULL OR wind_speed IS NOT NULL OR pm25_standard IS NOT NULL
ORDER BY ingested_at DESC LIMIT 1000
""").to_pandas()
print(f"Weather readings: {len(weather_df)}")
weather_df

In [ ]:
if not weather_df.empty and 'TEMPERATURE' in weather_df.columns and weather_df['TEMPERATURE'].notna().any():
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Temperature °C', 'Humidity %', 'Pressure hPa', 'Air Quality IAQ'))
    df = weather_df[weather_df['TEMPERATURE'].notna()]
    fig.add_trace(go.Scatter(x=df['INGESTED_AT'], y=df['TEMPERATURE'], mode='lines+markers', name='Temp °C'), row=1, col=1)
    if 'RELATIVE_HUMIDITY' in df.columns: fig.add_trace(go.Scatter(x=df['INGESTED_AT'], y=df['RELATIVE_HUMIDITY'], mode='lines+markers', name='Humidity'), row=1, col=2)
    if 'BAROMETRIC_PRESSURE' in df.columns: fig.add_trace(go.Scatter(x=df['INGESTED_AT'], y=df['BAROMETRIC_PRESSURE'], mode='lines+markers', name='Pressure'), row=2, col=1)
    if 'IAQ' in df.columns: fig.add_trace(go.Scatter(x=df['INGESTED_AT'], y=df['IAQ'], mode='lines+markers', name='IAQ'), row=2, col=2)
    fig.update_layout(height=600, title='Environmental Conditions')
    fig.show()
else:
    print("No temperature data yet - waiting for telemetry")

## Summary Stats

In [ ]:
stats = session.sql("""
SELECT COUNT(*) as total_packets, COUNT(DISTINCT from_id) as unique_nodes,
    MIN(ingested_at) as first_packet, MAX(ingested_at) as last_packet,
    COUNT(CASE WHEN latitude IS NOT NULL THEN 1 END) as gps_readings,
    COUNT(CASE WHEN temperature IS NOT NULL THEN 1 END) as temp_readings,
    COUNT(CASE WHEN relative_humidity IS NOT NULL THEN 1 END) as humidity_readings,
    COUNT(CASE WHEN barometric_pressure IS NOT NULL THEN 1 END) as pressure_readings,
    COUNT(CASE WHEN pm25_standard IS NOT NULL THEN 1 END) as air_quality_readings,
    ROUND(AVG(temperature), 2) as avg_temp_c, ROUND(AVG(relative_humidity), 2) as avg_humidity,
    ROUND(AVG(battery_level), 2) as avg_battery, ROUND(AVG(rx_snr), 2) as avg_snr
FROM DEMO.DEMO.MESHTASTIC_DATA
""").to_pandas()
for col in stats.columns: print(f"{col}: {stats[col].iloc[0]}")

## GPS Map

In [ ]:
pos_df = session.sql("""
SELECT from_id, latitude, longitude, altitude, ground_speed, sats_in_view, ingested_at
FROM DEMO.DEMO.MESHTASTIC_DATA WHERE latitude IS NOT NULL ORDER BY ingested_at DESC LIMIT 500
""").to_pandas()
if not pos_df.empty:
    fig = px.scatter_mapbox(pos_df, lat='LATITUDE', lon='LONGITUDE', color='FROM_ID',
        hover_data=['ALTITUDE', 'GROUND_SPEED', 'SATS_IN_VIEW'], zoom=10, height=500)
    fig.update_layout(mapbox_style='open-street-map', title='Node Positions')
    fig.show()

## Device Battery & Signal

In [ ]:
device_df = session.sql("""
SELECT from_id, ingested_at, battery_level, voltage, rx_snr, rx_rssi, channel_utilization, air_util_tx
FROM DEMO.DEMO.MESHTASTIC_DATA WHERE battery_level IS NOT NULL ORDER BY ingested_at DESC LIMIT 500
""").to_pandas()
if not device_df.empty:
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Battery %', 'Voltage V', 'SNR dB', 'RSSI dBm'))
    for node in device_df['FROM_ID'].unique()[:5]:
        ndf = device_df[device_df['FROM_ID']==node]
        fig.add_trace(go.Scatter(x=ndf['INGESTED_AT'], y=ndf['BATTERY_LEVEL'], name=node), row=1, col=1)
        fig.add_trace(go.Scatter(x=ndf['INGESTED_AT'], y=ndf['VOLTAGE'], showlegend=False), row=1, col=2)
        fig.add_trace(go.Scatter(x=ndf['INGESTED_AT'], y=ndf['RX_SNR'], showlegend=False), row=2, col=1)
        fig.add_trace(go.Scatter(x=ndf['INGESTED_AT'], y=ndf['RX_RSSI'], showlegend=False), row=2, col=2)
    fig.update_layout(height=600)
    fig.show()

## Export All Data

In [ ]:
all_df.to_csv('meshtastic_all_data.csv', index=False)
print(f"Exported {len(all_df)} rows to meshtastic_all_data.csv")